In [120]:
import numpy as np
import pandas as pd
import re
import os
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
##
stop_words.remove('no')
stop_words.remove('not')
from pickle import dump,load
import contractions
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

##
from keras import initializers
from keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

In [2]:
import gensim

C:\Users\Kaustav\Anaconda3\envs\keras\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
reviews = pd.read_csv('Reviews.csv')
reviews.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [6]:
print(reviews.isnull().sum())

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64


For 27 records summary is missing

In [7]:
reviews.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time'], axis =1, inplace = True)

reviews.dropna(inplace = True)
reviews.head(2)

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [8]:
for i in range(5):
    
    print('Text   :::::',reviews['Text'].iloc[i])
    print('Summary:::::',reviews['Summary'].iloc[i], '\n')
    

Text   ::::: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
Summary::::: Good Quality Dog Food 

Text   ::::: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
Summary::::: Not as Advertised 

Text   ::::: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat t

In [13]:
def clean_text(text, remove_stopwords = True):
    '''
    Preprocess the text 
    '''
    ##Will add to preprocess Class Later
    text = text.lower()
    ## Expand the contractions
    ## You can use a dictionary to do the same job. But I am lazy
    text = contractions.fix(text) 
    for word in text:
        text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags =re.MULTILINE)
        text = re.sub(r'\<a href',' ',text)
        text = re.sub(r'&amp;','',text)
        text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]',' ',text)
        text = re.sub(r'<br />',' ',text)
        text = re.sub(r'\'',' ',text)
    
    if remove_stopwords == True:
        text = text.split()
        text = ' '.join([word for word in text if word not in stop_words])
    
    return text
        

In [20]:


def get_clean_text(reviews):
    '''
    Clean the summaries and text
    '''
    # Clean the summaries
    clean_summaries = []
    
    ##for summary in reviews['Summary']:
    for summary in tqdm_notebook(reviews['Summary'], total= len(reviews['Summary'])):
        clean_summaries.append(clean_text(summary,remove_stopwords = False ))
    
    print(' Summaries clean processed')
    # Clean the texts
    text_reviews = []
    ##for single_text in reviews['Text']:
    for single_text in tqdm_notebook(reviews['Text'], total= len(reviews['Text'])):
        text_reviews.append(clean_text(single_text,remove_stopwords = True))
    
    print(' Text clean processed')
    return text_reviews, clean_summaries

text_reviews, clean_summaries = get_clean_text(reviews)


 Summaries clean processed



 Text clean processed


In [105]:
def store_clean_texts(text_reviews,clean_summaries):
    '''
    Store Clean Texts ans summaries
    '''
    #clean_df = pd.DataFrame()
    #clean_df['Summary'] = clean_summaries
    #clean_df['Text'] = text_reviews 
    #clean_df.to_csv('clean_df.csv', index= False)
    ###
    stories = list()
    for i, text in enumerate(text_reviews):
        stories.append({'story': text, 'highlights':clean_summaries[i] })
    
    # save to file
    dump(stories,open('D:\Project data\Text Summarizer\Amazon Food Review/review_dataset.pkl','wb'))
    print('Stored the Cleaned File')
    

store_clean_texts(text_reviews,clean_summaries)

Stored the Cleaned File


## Start from  here

In [148]:
def hyperparameters():
    batch_size = 64
    epochs = 10
    latent_dim = 256
    num_samples = 10000
    VOCAB_SIZE = 50000
    SUMMARY_VOCAB_SIZE = 46615
    text_pad_length = 117
    summary_pad_length = 9
    ##
    MAX_LEN = 1000
    VOCAB_SIZE =15000
    EMBEDDING_DIM = 300
    HIDDEN_UNITS = 200
    #VOCAB_SIZE = VOCAB_SIZE + 1

    LEARNING_RATE = 0.002
    BATCH_SIZE = 32
    EPOCHS = 5
    
    print('HyperParameter Set')
    return batch_size,epochs,latent_dim,num_samples, VOCAB_SIZE,text_pad_length,summary_pad_length,SUMMARY_VOCAB_SIZE,\
           LEARNING_RATE,BATCH_SIZE, EPOCHS


batch_size,epochs,latent_dim,num_samples, VOCAB_SIZE,text_pad_length,summary_pad_length,SUMMARY_VOCAB_SIZE,\
           LEARNING_RATE,BATCH_SIZE, EPOCHS = hyperparameters()

HyperParameter Set


In [5]:
def load_clean_texts(path='D:\Project data\Text Summarizer\Amazon Food Review', file_name='review_dataset.pkl'):
    stories = load(open(os.path.join(path, file_name), 'rb'))
    
    print('No of Stories Loaded', len(stories))
    return stories

#stories = load_clean_texts(path='D:\Project data\Text Summarizer\Amazon Food Review', file_name='review_dataset.pkl')

In [6]:

def get_text(path='D:\Project data\Text Summarizer\Amazon Food Review', file_name='review_dataset.pkl'):
    '''
    Load the Text and the Summaries to continue
    '''
    ##Load Stories
    stories = load_clean_texts(path='D:\Project data\Text Summarizer\Amazon Food Review', file_name='review_dataset.pkl')
    ##Stories loaded
    input_texts = []
    target_texts = []
    ##for story in stories:
    for story in tqdm_notebook(stories, total =len(stories)):
        input_text = story['story']
        target_text = story['highlights']
        
        input_texts.append(input_text)
        target_texts.append(target_text)
    ##    
    clean_df = pd.DataFrame()
    clean_df['Text'] = input_texts 
    clean_df['Summary'] = target_texts
    clean_df['Text_length']  = [len(x.split()) for x in clean_df['Text']]
    clean_df['Summary_length']  = [len(x.split()) for x in clean_df['Summary']]
    return clean_df
    
clean_df = get_text(path='D:\Project data\Text Summarizer\Amazon Food Review', file_name='review_dataset.pkl')

No of Stories Loaded 568427


In [7]:
print('Length of Text:')
print(clean_df['Text_length'].describe(percentiles = [0.05, .25, .5, .75, .95, .98, .99]))
##
print('\n')
print('Length of Summary:')
print(clean_df['Summary_length'].describe(percentiles = [0.05, .25, .5, .75, .95, .98, .99]))

Length of Text:
count    568427.000000
mean         42.196340
std          43.582674
min           0.000000
5%           10.000000
25%          17.000000
50%          29.000000
75%          51.000000
95%         117.000000
98%         167.000000
99%         211.000000
max        2114.000000
Name: Text_length, dtype: float64


Length of Summary:
count    568427.000000
mean          4.183987
std           2.660351
min           0.000000
5%            1.000000
25%           2.000000
50%           4.000000
75%           5.000000
95%           9.000000
98%          11.000000
99%          13.000000
max          48.000000
Name: Summary_length, dtype: float64


##### I will take the encoder length of 117 ( text length) and decoder length of 9 (Summary Length)

In [8]:
clean_df.drop(['Text_length', 'Summary_length'],axis = 1, inplace =True)


In [17]:
def get_text_and_summary(clean_df):
    input_texts = clean_df['Text']
    input_summary = clean_df['Summary']
    return input_texts,input_summary
##

input_texts,input_summary= get_text_and_summary(clean_df)

In [9]:
#### Tokenizer has some constraints. We may overcome it using Gensim. But Here I am using manual tokenization

In [75]:
seed =1987


def get_train_valid_set(clean_df):
    '''
    Split the Data to train and Validation Set
    '''
    from sklearn.utils import shuffle
    from sklearn.model_selection import train_test_split
    clean_df = shuffle(clean_df)
    
    ##
    input_texts,input_summary= get_text_and_summary(clean_df)
    train_input_texts = input_texts[10000:]
    valid_input_texts = input_texts[:10000]
    train_input_summary = input_texts[10000:]
    valid_input_summary = input_texts[:10000]
    return train_input_texts,valid_input_texts,train_input_summary,valid_input_summary


train_input_texts,valid_input_texts,train_input_summary,valid_input_summary = get_train_valid_set(clean_df)

In [76]:
clean_df.shape[0]

568427

In [77]:
len(train_input_texts)

558427

##### Seq2Seq Model

In [131]:
def tokenize_text_and_summary(train_input_texts,valid_input_texts,train_input_summary,valid_input_summary):
    text_tokenizer = Tokenizer(num_words=VOCAB_SIZE) ## Set by hyperparameters
    text_tokenizer.fit_on_texts(train_input_texts)
    ##print(len(text_tokenizer.word_index))
    X_train_seq = text_tokenizer.texts_to_sequences(train_input_texts)
    X_valid_seq = text_tokenizer.texts_to_sequences(valid_input_texts)
    ## I verified only 46615 words are there where word_count is > 3 for my data
    summary_tokenizer = Tokenizer(num_words=SUMMARY_VOCAB_SIZE) 
    summary_tokenizer.fit_on_texts(train_input_summary)
    ##print(len(text_tokenizer.word_index))
    ##len(tokenizer.word_index)
    X_train_summary_seq = summary_tokenizer.texts_to_sequences(train_input_summary)
    X_valid_summary_seq = summary_tokenizer.texts_to_sequences(valid_input_summary)
    return text_tokenizer,summary_tokenizer,X_train_seq,X_valid_seq,X_train_summary_seq,X_valid_summary_seq


text_tokenizer,summary_tokenizer,X_train_seq,X_valid_seq,X_train_summary_seq,X_valid_summary_seq = \
    tokenize_text_and_summary(train_input_texts,valid_input_texts,train_input_summary,valid_input_summary)

In [132]:
text_word_index_VOCAB_SIZE = {}
word_index = text_tokenizer.word_index
for word in word_index.keys():
    if word_index[word] == 0:
        print('Found 0!!')
    elif  word_index[word] > VOCAB_SIZE:
        continue
    else:
        text_word_index_VOCAB_SIZE[word] = word_index[word]
    
print(len(text_word_index_VOCAB_SIZE))

50000


In [134]:
summary_word_index_VOCAB_SIZE = {}
word_index = summary_tokenizer.word_index
for word in word_index.keys():
    if word_index[word] == 0:
        print('Found 0!!')
    elif  word_index[word] > SUMMARY_VOCAB_SIZE:
        continue
    else:
        summary_word_index_VOCAB_SIZE[word] = word_index[word]
    
print(len(summary_word_index_VOCAB_SIZE))

46615


In [135]:
summary_word_index = summary_tokenizer.word_index
print(len(summary_word_index))

109992


#### pad_sequences

In [136]:
from keras.preprocessing.sequence import pad_sequences

def pad_output(X_train_seq,X_valid_seq,X_train_summary_seq,X_valid_summary_seq):
    '''
    Pad the Text Sequences and Summary Sequences 
    '''
    X_train_seq_pad = pad_sequences(X_train_seq, maxlen = text_pad_length, padding = 'post', truncating = 'post')
    X_valid_seq_pad = pad_sequences(X_valid_seq, maxlen = text_pad_length, padding = 'post', truncating = 'post')
    X_train_summary_seq_pad = pad_sequences(X_train_summary_seq, maxlen = summary_pad_length, padding = 'post',\
                                            truncating = 'post')
    X_valid_summary_seq_pad = pad_sequences(X_valid_summary_seq, maxlen = summary_pad_length, padding = 'post',\
                                    truncating = 'post')
    return X_train_seq_pad,X_valid_seq_pad,X_train_summary_seq_pad,X_valid_summary_seq_pad


X_train_seq_pad,X_valid_seq_pad,X_train_summary_seq_pad,X_valid_summary_seq_pad = \
            pad_output(X_train_seq,X_valid_seq,X_train_summary_seq,X_valid_summary_seq)

In [137]:
X_train_seq_pad.shape
X_valid_seq_pad.shape
X_train_summary_seq_pad.shape
X_valid_summary_seq_pad.shape

(10000, 9)

In [138]:
##Pretrained glove300D

In [139]:
def load_pretrained_embedding(path= 'D:\Project data\glove_vectors\glove.6B.300d.txt', encoding = 'utf-8'):
    embeddings_index = {}
    with open(path, encoding = encoding) as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coef = np.asarray(values[1:], dtype = 'float32')
                embeddings_index[word] = coef
            except:
                continue

    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index




In [140]:
# Find the number of words that are missing from CN, and are used more than our threshold.
def  missing_word_analysis(text_word_index_VOCAB_SIZE,embeddings_index):
    missing_words = 0
    threshold = 15
    for word, count in text_word_index_VOCAB_SIZE.items():
        if count > threshold:
            if word not in embeddings_index:
                missing_words +=1
            
    missing_ratio = round(missing_words/len(text_word_index_VOCAB_SIZE),4)*100

    print("Number of words missing from CN:", missing_words)
    print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))
    
##
embeddings_index = load_pretrained_embedding(path= 'D:\Project data\glove_vectors\glove.6B.300d.txt', 
                                             encoding = 'utf-8')
missing_word_analysis(text_word_index_VOCAB_SIZE,embeddings_index)

Found 400000 word vectors.
Number of words missing from CN: 11847
Percent of words that are missing from vocabulary: 23.69%


In [141]:
missing_word_analysis(summary_word_index_VOCAB_SIZE,embeddings_index)

Number of words missing from CN: 10289
Percent of words that are missing from vocabulary: 22.07%


In [113]:
##Using conceptnet pretrained weights
embeddings_index = load_pretrained_embedding(path= 'D:\Project data\glove_vectors\conceptnet_numberbatch_en.txt', 
                                             encoding = 'utf-8')
missing_word_analysis(text_word_index_VOCAB_SIZE,embeddings_index)

Found 417195 word vectors.
Number of words missing from CN: 13835
Percent of words that are missing from vocabulary: 27.67%


In [129]:
#print(sorted(tokenizer.word_counts))
#print(tokenizer.document_count)
#print(tokenizer.word_index)
#print(tokenizer.word_docs)


In [143]:
def embedding_matrix_creater(embedding_dimention, word_index,embeddings_index):
    '''
    Build the embedding matrix using pretrained weights
    Use zeros if the word is not there in pretrained vectors
    '''
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, index in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embedding_matrix


##
embedding_dimention = 300
text_embedding_matrix   = embedding_matrix_creater(embedding_dimention, text_word_index_VOCAB_SIZE,embeddings_index)
print(text_embedding_matrix.shape)
summary_embedding_matrix= embedding_matrix_creater(embedding_dimention, summary_word_index_VOCAB_SIZE,embeddings_index)
print(summary_embedding_matrix.shape)

(50001, 300)
(46616, 300)


In [121]:
encoder_embedding_layer = Embedding(input_dim = 50001, 
                                    output_dim = 300,
                                    input_length = text_pad_length, ###MAX_LEN,
                                    weights = [text_embedding_matrix],
                                    trainable = False)


In [123]:
decoder_embedding_layer = Embedding(input_dim = 109993, 
                                    output_dim = 300,
                                    input_length = summary_pad_length, ##MAX_LEN,
                                    weights = [summary_embedding_matrix],
                                    trainable = False)

In [150]:
# Hyperparams

HIDDEN_UNITS = 200

### Model 1. Simple LSTM Encoder-Decoder-seq2seq

In [152]:
"""
Simple LSTM Encoder-Decoder-seq2seq
"""
# encoder
encoder_inputs = Input(shape=(text_pad_length, ), dtype='int32',)
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS)(encoder_embedding)
# decoder
decoder_inputs = Input(shape=(summary_pad_length, ))
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS)(decoder_embedding)
# merge
merge_layer = concatenate([encoder_LSTM, decoder_LSTM])
decoder_outputs = Dense(units=SUMMARY_VOCAB_SIZE+1, activation="softmax")(merge_layer) # SUM_VOCAB_SIZE, sum_embedding_matrix.shape[1]

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 117)          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 117, 300)     15000300    input_4[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 9, 300)       32997900    input_5[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [153]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train_seq_pad,X_valid_seq_pad,X_train_summary_seq_pad,X_valid_summary_seq_pad

In [ ]:

history = model.fit([X_train_seq_pad, X_train_summary_seq_pad], 
                     target_train, 
                     epochs=EPOCHS, 
                     batch_size=BATCH_SIZE,
                     validation_data=([X_valid_seq_pad, X_valid_summary_seq_pad], target_test))

### Model 2. Bidirectional LSTM Encoder-Decoder-seq2seq

In [ ]:

"""
Bidirectional LSTM: Others Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LEN,))
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_LSTM_R = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

"""
decoder
"""
decoder_inputs = Input(shape=(MAX_LEN,))
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 
decoder_dense = Dense(VOCAB_SIZE, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

model= Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

#### Model 3. Chatbot Inspired Encoder-Decoder-seq2seq

In [ ]:
"""
Chatbot Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs)

In [ ]:
rmsprop = RMSprop(lr=0.01, clipnorm=1.)
model.compile(loss='mse', optimizer=rmsprop, metrics=["accuracy"])

In [ ]:

# model 2
model.summary()

### Alternate 1: One-Shot Model

In [ ]:
src_txt_length = ...
sum_txt_length = ...
# encoder input model
inputs = Input(shape=(src_txt_length,))
encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(encoder1)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')